In [ ]:
import requests
import pandas as pd
import mysql.connector
from collections import Counter
from datetime import datetime

# extraer de la api
url_base = "https://mnpwhdbcsk.us-east-2.awsapprunner.com/api"
headers = {"x-api-key": "8yBO1wKiiIbcBT0"}

data_products  = requests.get(f"{url_base}/products",  headers=headers).json().get("data", [])
data_purchases = requests.get(f"{url_base}/purchases", headers=headers).json().get("data", [])

# transformacion
precios = {p["id"]: float(p["price"]) for p in data_products}

for c in data_purchases:
    c["total"] = round(sum(precios.get(p["id"], 0) * (1 - p.get("discount", 0)/100) for p in c["products"]), 2)

purchase_products_data = []
for c in data_purchases:
    cnt = Counter([p["id"] for p in c["products"]])
    for pid, qty in cnt.items():
        purchase_products_data.append({"purchase_id": c["id"], "product_id": pid, "quantity": qty})

for p in data_products:
    if isinstance(p.get("price"), int):
        p["price"] = float(p["price"])
    if "createdAt" in p:
        p["created_at"] = p.pop("createdAt")
    try:
        p["created_at"] = datetime.strptime(p["created_at"], "%m/%d/%Y").strftime("%Y-%m-%d")
    except:
        p["created_at"] = None

for c in data_purchases:
    try:
        c["purchaseDate"] = datetime.strptime(c["purchaseDate"], "%m/%d/%Y").strftime("%Y-%m-%d")
    except:
        c["purchaseDate"] = None

# dfs
df_products  = pd.DataFrame(data_products)[["id","name","description","price","category","created_at"]]
df_purchases = pd.DataFrame(data_purchases)[["id","status","creditCardType","purchaseDate","total"]]
df_pp        = pd.DataFrame(purchase_products_data)

# claves como texto 
df_products["id"] = df_products["id"].astype(str)
df_purchases["id"] = df_purchases["id"].astype(str)
if not df_pp.empty:
    df_pp["purchase_id"] = df_pp["purchase_id"].astype(str)
    df_pp["product_id"]  = df_pp["product_id"].astype(str)

df_products  = df_products.where(pd.notnull(df_products),  None)
df_purchases = df_purchases.where(pd.notnull(df_purchases), None)
df_pp        = df_pp.where(pd.notnull(df_pp),               None)

rows_products  = list(df_products.itertuples(index=False, name=None))
rows_purchases = list(df_purchases.itertuples(index=False, name=None))
rows_pp        = list(df_pp.itertuples(index=False, name=None))

# data warehouse 
conexion = mysql.connector.connect(
    host="35.231.201.180",
    user="integrationISP",
    password="e/>E2$Z1jxX-%GhY",
    database="take_home_project"
)
cur = conexion.cursor()

sql_products = """
INSERT INTO products (id,name,description,price,category,created_at)
VALUES (%s,%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
  name=VALUES(name),
  description=VALUES(description),
  price=VALUES(price),
  category=VALUES(category),
  created_at=VALUES(created_at)
"""

sql_purchases = """
INSERT INTO purchases (id,status,credit_card_type,purchase_date,total)
VALUES (%s,%s,%s,%s,%s)
ON DUPLICATE KEY UPDATE
  status=VALUES(status),
  credit_card_type=VALUES(credit_card_type),
  purchase_date=VALUES(purchase_date),
  total=VALUES(total)
"""

sql_pp = """
INSERT INTO purchase_products (purchase_id,product_id,quantity)
VALUES (%s,%s,%s)
ON DUPLICATE KEY UPDATE
  quantity=VALUES(quantity)
"""

batch_size = 1000

def insert_batches(sql, rows):
    for i in range(0, len(rows), batch_size):
        cur.executemany(sql, rows[i:i+batch_size])

try:
    insert_batches(sql_products,  rows_products)
    insert_batches(sql_purchases, rows_purchases)
    insert_batches(sql_pp,        rows_pp)
    conexion.commit()
finally:
    cur.close()
    conexion.close()
